In [159]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
import glob
from scipy.signal import butter
import scipy
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from scipy import stats
import os
from scipy.sparse import spdiags
import pandas as pd
FS = 60

## FFT Heart Rate

In [160]:
def next_power_of_2(x):
    return 1 if x == 0 else 2**(x - 1).bit_length()

def calculate_fft_hr(ppg_signal, fs=60):
    ppg_signal = np.expand_dims(ppg_signal, 0)
    N = next_power_of_2(ppg_signal.shape[1])
    f_ppg, pxx_ppg = scipy.signal.periodogram(ppg_signal, fs=fs, nfft=N, detrend=False)
    fmask_ppg = np.argwhere((f_ppg >= 0.75) & (f_ppg <= 2.5))
    mask_ppg = np.take(f_ppg, fmask_ppg)
    mask_pxx = np.take(pxx_ppg, fmask_ppg)
    fft_hr = np.take(mask_ppg, np.argmax(mask_pxx, 0))[0] * 60
    return fft_hr, mask_ppg, mask_pxx


### Detrend signal

In [161]:
def detrend(signal, Lambda):
    signal_length = signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    # second-order difference matrix
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = spdiags(diags_data, diags_index, (signal_length - 2), signal_length).toarray()
    filtered_signal = np.dot((H - np.linalg.inv(H + (Lambda ** 2) * np.dot(D.T, D))), signal)
    return filtered_signal


### Remove artifacts

In [162]:
def remove_artifact(ppg):
    ppgmean = np.mean(ppg)
    ppgstddev = np.std(ppg[:200])
    f1 = np.where(ppg > (ppgmean + 4 * ppgstddev))[0]
    f2 = np.where(ppg < (ppgmean - 4 * ppgstddev))[0]
    f = np.array(np.union1d(f1, f2))
    replace_value = np.array([ppgmean]*f.shape[0])
    np.put(ppg, f, replace_value)
    return ppg

### Filter

In [163]:
def filter_ppg(ppg, fs=60, lpf=0.4, hpf=2.5):    
    [b_pulse, a_pulse] = butter(1, [lpf / fs * 2, hpf / fs * 2], btype='bandpass')
    return scipy.signal.filtfilt(b_pulse, a_pulse, np.double(ppg))

## Normalization

In [164]:
def normalize_ppg(ppg):
    ppg = ppg - np.min(ppg)
    ppg = ppg / np.max(ppg)
    return ppg

### Find peaks

In [165]:
def find_peaks(ppg, height=0.6):
    peaks, _ = scipy.signal.find_peaks(ppg, height=height, distance=20)
    return peaks


def load_ecg_peaks(csv_path):
    df = pd.read_csv(csv_path)
    qrs = np.array(df['qrs_detected'].tolist()).astype(np.int)
    qrs_indices = np.where(qrs == 1) 
    return qrs_indices


def load_ecg_qrs(csv_path):
    df = pd.read_csv(csv_path)
    qrs = np.array(df['qrs_detected'].tolist()).astype(np.int)
    return qrs


### Remove ectopic beats

In [166]:
def remove_ectopic(ppg_peaks):
    rr_intervals_list = np.diff(ppg_peaks)
    # # This remove ectopic beats from signal
    nn_intervals_list = remove_ectopic_beats(rr_intervals=rr_intervals_list, method="malik")
    # This replace ectopic beats nan values with linear interpolation
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)
    return np.cumsum(interpolated_nn_intervals).astype(np.int)

## Plot func

In [167]:
def plot_peaks(test_face_ppg, test_ppg, test_ecg, peaks_face_ppg, peaks_ppg, peaks_ecg):
    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_face_ppg)
    plt.plot(peaks_face_ppg, test_face_ppg[peaks_face_ppg], "x")
    plt.title('Face')
    plt.show()

    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_ppg)
    plt.plot(peaks_ppg, test_ppg[peaks_ppg], "x")
    plt.title('Finger')
    plt.show()
    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_ecg)
    plt.plot(peaks_ecg, test_ecg[peaks_ecg], "x")
    plt.title('ECG')
    plt.show()

def plot_signal(test_face_ppg, test_ppg, test_ecg, finger_hr_fft, face_hr_fft, ecg_hr_fft, 
                ecg_peaks, subj_name, peak_plot=True):
    ecg_peaks = np.asarray(ecg_peaks)    
    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_face_ppg)
    plt.title('Face: ' + str(face_hr_fft) + ' Sub: ' + subj_name)
    plt.show()

    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_ppg)
    plt.title('Finger: ' + str(finger_hr_fft) + ' Sub: ' + subj_name)
    plt.show()

    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(test_ecg)
    if peak_plot:
        plt.plot(ecg_peaks, test_ecg[ecg_peaks], "x", color='r')
    plt.title('ECG: ' + str(ecg_hr_fft) + ' Sub: ' + subj_name)
    plt.show()

def plot_fft(mask_ppg, mask_pxx, title):
    plt.plot(mask_ppg, np.reshape(mask_pxx, [-1, 1]))
    plt.title(f'{title} FFT')
    plt.show()


### Calculate IBI

In [168]:
'''
1. detrend the finger ppg. 
2. filter and clean the finger ppg.
3. calculate ecg hr vs. finger ppg hr vs. face hr.
4. sync video using rr-interval. Max deday should be around 10s. 
'''

'\n1. detrend the finger ppg. \n2. filter and clean the finger ppg.\n3. calculate ecg hr vs. finger ppg hr vs. face hr.\n4. sync video using rr-interval. Max deday should be around 10s. \n'

In [169]:
def calcuate_window_ibi(finger_ppg, face_ppg, ecg, window_size=5, fs=60):
#     finger_ppg = detrend(finger_ppg, 100)
    face_ppg = normalize_ppg(filter_ppg(remove_artifact(face_ppg)))
    finger_ppg = normalize_ppg(filter_ppg(remove_artifact(finger_ppg)))
#     ecg = normalize_ppg(ecg)
    finger_hr = []
    face_hr = []
    ecg_hr = []
    for idx in range(0, len(finger_ppg)-fs*window_size, fs*window_size):
        window_finger_ppg = finger_ppg[idx:idx + fs * window_size]
        window_face_ppg = face_ppg[idx:idx + fs * window_size]
        window_ecg = ecg[idx:idx + fs * window_size]
        window_peaks_finger_ppg = find_peaks(window_finger_ppg)
        window_peaks_face_ppg = find_peaks(window_face_ppg)
        window_peaks_ecg = ecg_detectors.hamilton_detector(window_ecg)
        window_avg_hr_finger = 60 / (np.mean(np.diff(window_peaks_finger_ppg)) / fs)
        window_avg_hr_face = 60 / (np.mean(np.diff(window_peaks_face_ppg)) / fs)
        window_avg_hr_ecg = 60 / (np.mean(np.diff(window_peaks_ecg)) / fs)
        finger_hr.append(window_avg_hr_finger)
        face_hr.append(window_avg_hr_face)
        ecg_hr.append(window_avg_hr_ecg)
#         if abs(window_avg_hr_face - window_avg_hr_ecg) > 5:
#             plot_peaks(window_face_ppg, window_finger_ppg, window_ecg, window_peaks_face_ppg, 
#                        window_peaks_finger_ppg, window_peaks_ecg)
#             print('window_avg_hr_face: ', window_avg_hr_face)
#             print('window_avg_hr_finger: ', window_avg_hr_finger)
#             print('window_avg_hr_ecg: ', window_avg_hr_ecg)
    return finger_hr, face_hr, ecg_hr


def calcuate_window_ibi_fft(finger_ppg, face_ppg, ecg, qrs_peaks, subj_name, window_size=5, fs=60):
#     finger_ppg = detrend(finger_ppg, 100)
    face_ppg = normalize_ppg(filter_ppg(remove_artifact(face_ppg)))
    finger_ppg = normalize_ppg(filter_ppg(remove_artifact(finger_ppg)))
    ecg = normalize_ppg(ecg)
    finger_hr = []
    face_hr = []
    ecg_hr = []
    for idx in range(0, len(finger_ppg), fs*window_size):
        window_finger_ppg = finger_ppg[idx:idx + fs * window_size]
        window_face_ppg = face_ppg[idx:idx + fs * window_size]
        window_ecg = ecg[idx:idx + fs * window_size]
        window_avg_hr_finger, _, _ = calculate_fft_hr(window_finger_ppg, fs=60)
        window_avg_hr_face, _, _ = calculate_fft_hr(window_face_ppg, fs=60)
        window_ecg_qrs = qrs_peaks[idx:idx + fs * window_size]
        window_ecg_qrs = np.where(window_ecg_qrs == 1) 
        window_avg_hr_ecg = 60 / (np.mean(np.diff(window_ecg_qrs)) / fs)
        finger_hr.append(window_avg_hr_finger)
        face_hr.append(window_avg_hr_face)
        ecg_hr.append(window_avg_hr_ecg)
#         if abs(window_avg_hr_face - window_avg_hr_ecg) > 10:
#             plot_signal(window_face_ppg, window_finger_ppg, window_ecg, 
#                         window_avg_hr_finger, window_avg_hr_face, window_avg_hr_ecg, 
#                         window_ecg_qrs, subj_name, peak_plot=True)
    return finger_hr, face_hr, ecg_hr


def metrics_calculation(gt_hr, est_hr):
    gt_hr, est_hr = np.array(gt_hr), np.array(est_hr)
    mae = np.mean(np.abs(gt_hr - est_hr))
    rmse = np.sqrt(np.mean((gt_hr - est_hr)**2))
    mape = np.mean(np.abs(gt_hr - est_hr) / gt_hr)
    return mae, rmse, mape

In [170]:
all_mat_path = sorted(glob.glob("../ProcessedDataNoVideo/*.mat"))
# bad_subject = ['P032a', 'P032b', 'P038a', 'P038b', 'P055a', 'P055b', 'P068a', 'P068b']
bad_subject = ['P038a', 'P038b', 'P068a', 'P068b']
ecg_peak_version = 'v2'
window_size_all = [10, 15, 30, 60]
# window_size_all = [30]

for window_size in window_size_all:
    print('Window size: ', window_size)
    finger_hr_all = []
    face_hr_all = []
    for idx, mat_path in enumerate(all_mat_path):
        subj_name = os.path.basename(mat_path).split('.')[0]
        if subj_name in bad_subject:
            continue
        mat = scipy.io.loadmat(mat_path)
        face_ppg = mat['ppg_face'][0]
        finger_ppg = 1 - mat['ppg'][0] 
        ecg = mat['ekg'][0]
        csv_path = f"../ecg_detection/ecg_peaks_{ecg_peak_version}/{subj_name}.csv"
        qrs_peaks = load_ecg_peaks(csv_path)
        qrs = load_ecg_qrs(csv_path)
    #     finger_hr, face_hr, ecg_hr = calcuate_window_ibi(finger_ppg, face_ppg, ecg, window_size=window_size, fs=60)
    #     finger_hr, face_hr, ecg_hr = calcuate_video_hr_fft(finger_ppg, face_ppg, ecg, qrs_peaks, subj_name, fs=60)    
        finger_hr, face_hr, ecg_hr = calcuate_window_ibi_fft(finger_ppg, face_ppg, ecg, qrs, subj_name, 
                                                             window_size=window_size, fs=60)    

        mae, rmse, mape = metrics_calculation(finger_hr, face_hr)
        finger_hr_all.extend(finger_hr)
        face_hr_all.extend(face_hr)
    finger_hr_all, face_hr_all = np.array(finger_hr_all), np.array(face_hr_all)
    np.save(f'../results/finger_hr_all_ws_{str(window_size)}.npy', finger_hr_all)
    np.save(f'../results/face_hr_all_ws_{str(window_size)}.npy', face_hr_all)

Window size:  10
Window size:  15
Window size:  30
Window size:  60


### Test

In [171]:
# all_mat_path = sorted(glob.glob("./ProcessedDataNoVideo/*.mat"))
# mat = scipy.io.loadmat(all_mat_path[12])
# test_face_ppg = mat['ppg_face'][0]
# test_ppg = 1 - mat['ppg'][0]
# test_ppg = detrend(test_ppg, 100)
# test_ecg = mat['ekg'][0]
# cutoff = 3000
# test_face_ppg = normalize_ppg(filter_ppg(remove_artifact(test_face_ppg)))[:cutoff]
# test_ppg = normalize_ppg(filter_ppg(remove_artifact(test_ppg)))[:cutoff]
# test_ecg = normalize_ppg(test_ecg[:cutoff])
# peaks_face_ppg = find_peaks(test_face_ppg)
# peaks_ppg = find_peaks(test_ppg)
# # peaks_ecg = find_peaks(test_ecg)
# ecg_detector = QRSDetectorOffline(test_ecg, fs=60, 
#                                   integration_window=int((15/250)*60), 
#                                   refractory_period=int((120/250)*60), 
#                                   findpeaks_spacing=int((50/250)*60))
# peaks_ecg = ecg_detector.detect_qrs()
# print(peaks_ecg)
# plot_peaks(test_face_ppg, test_ppg, test_ecg, peaks_face_ppg, peaks_ppg, peaks_ecg)
# raise